In [3]:
!pip install googletrans==4.0.0=rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement googletrans==4.0.0=rc1 (from versions: 2.0.0, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0, 2.3.0, 2.4.0, 3.0.0, 3.1.0a0, 4.0.0rc1)
ERROR: No matching distribution found for googletrans==4.0.0=rc1


In [7]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 69.2 MB/s 


In [8]:
import pandas as pd
import pickle
import numpy as np
import os
from keras.applications.resnet import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from tensorflow.keras.models import Sequential, Model
from keras.utils import np_utils
import random
from tensorflow.keras.preprocessing import image, sequence
import matplotlib.pyplot as plt
from tqdm import tqdm
from zipfile import ZipFile
import tensorflow as tf
from IPython.display import Image, display
from google.colab import drive
import googletrans
from googletrans import Translator
from konlpy.tag import Okt

In [10]:
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
data="/content/drive/Shareddrives/투빅스_추천/1. 데이터/caption/archive (3).zip"

In [12]:
with open('/content/drive/Shareddrives/투빅스_추천/1. 데이터/caption/w2i.p', 'rb') as f:
    word_2_indices= pickle.load(f, encoding="bytes")
with open('/content/drive/Shareddrives/투빅스_추천/1. 데이터/caption/i2w.p', 'rb') as f:
    indices_2_word= pickle.load(f, encoding="bytes")

In [13]:
with ZipFile(data,"r") as zip1:
  zip1.extractall()

In [14]:
captions_path = '/content/Flickr_Data/Flickr_TextData/Flickr8k.token.txt'
captions = open(captions_path, 'r').read().split("\n")

In [15]:
model = tf.keras.models.load_model('/content/drive/Shareddrives/투빅스_추천/1. 데이터/caption/best_model_acc (1).h5')

In [16]:
def preprocessing(img_path):
    im = image.load_img(img_path, target_size=(224,224,3))
    im = image.img_to_array(im)
    im = np.expand_dims(im, axis=0)
    return im

def get_encoding(model, img):
    image = preprocessing(img)
    pred = model.predict(image).reshape(2048)
    return pred

In [17]:
max_len = 40

In [18]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')

94781440/94765736 [==============================] - 0s 0us/step


In [45]:
def predict_captions(image):
    start_word = ["<start>"]
    while True:
        par_caps = [word_2_indices[i] for i in start_word]
        par_caps = sequence.pad_sequences([par_caps], maxlen=40, padding='post')
        preds = model.predict([np.array([image]), np.array(par_caps)])
        word_pred = indices_2_word[np.argmax(preds[0])]
        start_word.append(word_pred)
        
        if word_pred == "<end>" or len(start_word) > 40:
            break
            
    return ' '.join(start_word[1:-1])

In [47]:
img = '/content/drive/Shareddrives/투빅스_추천/1. 데이터/caption/love.jpg'
test_img = get_encoding(resnet, img)

In [48]:
#캡셔닝 모델 돌리기
Argmax_Search = predict_captions(test_img)
#이미지 출력
z = Image(filename=img)
display(z)

print(Argmax_Search)

IndexError: ignored

## 태그 번역

In [ ]:
eng_sen = Argmax_Search

In [ ]:
translator =Translator()
result = translator.translate(eng_sen, dest="ko")
result.text

In [ ]:
okt = Okt()

In [ ]:
#명사
nouns = okt.nouns(result.text)
nouns

In [ ]:
#품사태그
pos = okt.pos(result.text,norm=True)
pos

In [ ]:
tag = ['Adjective','Noun','Adverb']
use_word = []
for i in range(len(pos)):
  if pos[i][1] in tag:
    use_word.append(pos[i][0])
use_word